# Results of LeNet Trained on MNIST

In this notebook I display results in terms of accuracy on test data for LeNet models trained on the MNIST dataset using various regularization techniques. The LeNet model and the MNIST dataset were set up as in Hoffman 2019 (for all the models). The MNIST data is preprocessed by normalizing using mean 0.1307 and variance 0.3081. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization, both with and without dropout with a dropout rate of p_drop = 0.5. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 250 epochs.

### Imports and Model Loading

In [ ]:
import jupyter_black
import numpy as np
import pandas as pd
import torch
from scipy.stats import sem, t
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_MNIST
from model_classes import LeNet_MNIST
from plotting_tools import get_random_img, generate_random_vectors
from tools import accuracy, compute_total_variation, ModelInfo

jupyter_black.load()

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_MNIST()

# Summary of model
summary_model = LeNet_MNIST().to(device)
summary(summary_model, (1, 28, 28))

In [ ]:
# Load models

# Defining model types and versions
model_types = ["no_reg", "l2", "jacobi", "svb"]
model_versions = ["", "_no_dropout"]

model_names = []

for t in model_types:
    for v in model_versions:
        for i in range(5):
            model_names.append(f"model_{t}{v}_{i}")

models = {name: ModelInfo(name) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [ ]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for t in model_types:
    for v in model_versions:
        accuracies = []
        for i in range(5): # Assuming 5 versions of each model
            model_name = f"model_{t}{v}_{i}"
            model = models[model_name].model
            acc = accuracy(model, test_loader, device)
            accuracies.append(acc)
        
        # Calculate mean accuracy
        mean_accuracy = np.mean(accuracies)
        
        # Calculate standard error
        std_error = sem(accuracies)
        
        # Calculate confidence interval
        CI = std_error * t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

        # Store results in the data dictionary
        accuracy_data[f"{t}{v}"] = (mean_accuracy, CI)

In [ ]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"])

# Reshape the DataFrame to have model types as rows and model versions as columns
df.index = pd.MultiIndex.from_tuples(df.index.str.split("_", expand=True).tolist(), names=["Type", "Version"])
df = df.unstack().swaplevel(axis=1).sort_index(axis=1)

print(df)

### Total Variation

Total variation is the sum of the absolute differences for neighboring pixel-values in the input images. This measures how much noise is in the images. I generate 25 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level.

In [ ]:
# Define zoom levels, number of images, and confidence level for confidence interval calculation
zoom_levels = [0.025, 0.01, 0.001]
n_images = 25
confidence_level = 0.95

# Dataframe to store results
df_results = pd.DataFrame(index=model_names, columns=zoom_levels)

# Loop over all models
for name in tqdm(model_names):
    model = models[name].model # Get model
    model.to(device)
    tv_values = {zoom: [] for zoom in zoom_levels} # To store total variation values for each zoom level

    # Genereate tv values for n_images number of images
    for _ in range(n_images):
        # Get random image and vectors
        img = get_random_img(test_loader)
        v1, v2 = generate_random_vectors(img)

        # Compute the total variation of decision boundaries for the current model 
        # and the generated image at different zoom levels
        tv_list = compute_total_variation(model, img, v1, v2, device)
        for zoom, tv in zip(zoom_levels, tv_list):
            tv_values[zoom].append(tv)
    
    # Calculate mean and 95% confidence interval for total variation values at each zoom level
    for zoom in zoom_levels:
        mean = np.mean(tv_values[zoom])
        std_err = np.std(tv_values[zoom]) / np.sqrt(n_images)
        conf_interval = t.ppf((1 + confidence_level) / 2, n_images - 1) * std_err
        df_results.loc[model_name, zoom] = f"{mean:.2f} +/- {conf_interval:.2f}" # Store results in dataframe

print(df_results)